In [215]:
## Stardew Valley Optimization


# ~/Library/gurobi900
# python3 setup.py install 
import gurobipy as gp
from gurobipy import GRB


In [212]:
crops, cost, yeild_time, regrow, sell_price = gp.multidict({
    'blue_jaz': [30, 7, None, 50],
    'cauliflower': [80, 12, None, 175],
    'green_bean': [60, 10, 3, 40],
    'kale': [70, 6, None, 110],
    'parsnip': [20,4, None, 35],
    'potato': [50,6,None,80],
    'rhubarb': [100,13,None,220],
    'strawberry': [100,8,4,120],
    'tulip': [20,6,None,30],
    'unmilled_rice': [40,6,None,30]
})


In [11]:
crops

['blue_jaz', 'cauliflower', 'green_bean', 'kale', 'parsnip', 'potato', 'rhubarb', 'strawberry', 'tulip', 'unmilled_rice']

In [3]:
def max_range_regrow(current_day, crop):
    if(current_day < yeild_time[crop]):
        return 0
    elif(current_day < yeild_time[crop] + regrow[crop]):
        return 1
    else:
        regrow_time = yeild_time[crop] + regrow[crop]
        count = 1
        while(current_day >= regrow_time):
            count +=1
            regrow_time += regrow[crop]
        return count

In [14]:
import pandas as pd 
stardew_map = pd.read_csv("stardew_valley_map.csv") 
stardew_map = stardew_map.fillna(0)
stardew_np = stardew_map.to_numpy()
copy = stardew_np.copy()

In [15]:
copy = copy.transpose()

In [16]:
copy

array([['G', 0, 0, ..., 0, 0, 'R'],
       ['G', 'T', 0, ..., 'R', 'R', 0],
       ['G', 0, 0, ..., 0, 0, 'R'],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [107]:
import numpy as np

In [437]:
copy = np.zeros( (29, 29) )


## going to add in the sprinkler system

In [438]:
# Create a new model
m = gp.Model("matrix1")
m.Params.MIPFocus = 3
m.Params.MIPGap = .011
# Create variables
purchase =  []
planted = []
harvested = []
stored = []
money_in_days = []
amount_growing = []
didPlant = []
soilPrepped = []
energySpent = []
occupied = []
greenBeanPlantedHere = []
strawberryPlantedHere = []
sprinklersPurchased = []
sprinklerPutHere = []
sprinklerCovered = []
sprinklersOwned = []
starting_parsnips = []
no_water_needed = []
time_spent = []
energy_penalty = []
bought_plants_today = []
bought_sprinkler_today = []
days = 28
starting_money = 500
for i in range(days):
    bought_sprinkler_today.append(m.addVar(vtype=GRB.INTEGER, name='bought sprinkler on ' + str(i)))
    bought_plants_today.append(m.addVar(vtype=GRB.INTEGER, name='bought plants on ' + str(i)))
    occupied.append([])
    time_spent.append(m.addVar(name='time_spent on ' + str(i)))
    energy_penalty.append(m.addVar(name='energy_penalty on ' + str(i)))
    sprinklersOwned.append(m.addVar(vtype=GRB.INTEGER, name='sprinklers owned'))
    sprinklerCovered.append([])
    no_water_needed.append([])
    starting_parsnips.append(m.addVars(crops, name='started with'))
    sprinklersPurchased.append(m.addVar(vtype=GRB.INTEGER,name='sprinkler purchased on' + str(i)))
    sprinklerPutHere.append([])
    money_in_days.append(m.addVar(name='money on ' + str(i)))
    purchase.append(m.addVars(crops,name='purchased on ' + str(i), vtype=GRB.INTEGER))
    planted.append(m.addVars(crops,name='planted on ' +str(i), vtype=GRB.INTEGER))
    harvested.append(m.addVars(crops, name='harvested on ' +str(i), vtype=GRB.INTEGER))
    stored.append(m.addVars(crops, name='stored on ' +str(i), vtype=GRB.INTEGER))
    amount_growing.append(m.addVar(name='amount growing on ' +str(i), vtype=GRB.INTEGER))
    energySpent.append(m.addVar(name='energy spent on ' + str(i)))
    didPlant.append([])
    soilPrepped.append([])
    strawberryPlantedHere.append([])
    greenBeanPlantedHere.append([])
    for x in range(copy.shape[0]):
        sprinklerCovered[i].append([])
        no_water_needed[i].append([])
        sprinklerPutHere[i].append([])
        greenBeanPlantedHere[i].append([])
        strawberryPlantedHere[i].append([])
        occupied[i].append([])
        didPlant[i].append([])
        soilPrepped[i].append([])
        for y in range(copy.shape[1]):
            sprinklerCovered[i][x].append(m.addVar(vtype=GRB.BINARY, name='sprink_covered on '+ 
                                        str(i) + " at (" + str(x) +  ',' + str(y) +')'))
            no_water_needed[i][x].append(m.addVar(vtype=GRB.BINARY, name='no_water on '+ 
                                        str(i) + " at (" + str(x) +  ',' + str(y) +')'))
            sprinklerPutHere[i][x].append(m.addVar(vtype=GRB.BINARY, name='sprinkler on '+ 
                                        str(i) + " at (" + str(x) +  ',' + str(y) +')'))
            greenBeanPlantedHere[i][x].append(m.addVar(vtype=GRB.BINARY))
            strawberryPlantedHere[i][x].append(m.addVar(vtype=GRB.BINARY))
            occupied[i][x].append(m.addVar(name='occupied on ' + 
                                        str(i) + " at (" + str(x) +  ',' + str(y) +')'))
            didPlant[i][x].append(m.addVars(crops, vtype=GRB.BINARY, name="did plant on " + 
                                        str(i) + " at (" + str(x) +  ',' + str(y) + ')'))
            soilPrepped[i][x].append(m.addVar(vtype=GRB.BINARY, name="soil prepped on " + 
                                        str(i) + " at (" + str(x) +  ',' + str(y) + ')'))

    
m.setObjective(money_in_days[days-1], GRB.MAXIMIZE)
    
for i in range(days):
    if(i==0):
        m.addConstr(gp.quicksum(starting_parsnips[i][j] for j in crops) == 15)
        m.addConstr(starting_parsnips[i]['parsnip'] == 15)
        m.addGenConstrIndicator(bought_plants_today[i], 0, gp.quicksum(purchase[i][j] for j in crops) == 0)
        m.addGenConstrIndicator(bought_plants_today[i], 1, gp.quicksum(purchase[i][j] for j in crops) >= 1)
        m.addGenConstrIndicator(bought_sprinkler_today[i], 0, sprinklersPurchased[i] == 0)
        m.addGenConstrIndicator(bought_sprinkler_today[i], 1, sprinklersPurchased[i] >= 1)
        ## on day 0 i can spend up to starting_money
        m.addConstr(money_in_days[i] == starting_money - gp.quicksum(purchase[i][j]*cost[j] for j in crops) 
                    - 1125 * sprinklersPurchased[i])
        ## have to water the crops you plant unless you have the sprinklers
        m.addConstr(gp.quicksum(soilPrepped[i][x][y] * 2
                                for x in range(copy.shape[0])
                                for y in range(copy.shape[1]))
                    + 2 * gp.quicksum(planted[i][j] for j in crops) 
                    - 2 * gp.quicksum(no_water_needed[i][x][y] 
                                  for x in range(copy.shape[0])
                                  for y in range(copy.shape[1]))
                    == energySpent[i])
        
        ## have to water and plant so it's 4
        m.addConstr(time_spent[i] == 
                    4 * gp.quicksum(planted[i][j] for j in crops) 
                    + 2 * gp.quicksum(soilPrepped[i][x][y]
                                for x in range(copy.shape[0])
                                for y in range(copy.shape[1]))
                   + 24 * bought_plants_today[i]
                   + 40 * bought_sprinkler_today[i])
        
        m.addConstr(time_spent[i] <= 810 + energy_penalty[i])
        
        m.addConstr(energySpent[i] <= 270)
        
        ## on day 0 you can't harvest anything, what you store is what you buy and not plant
        ## what you plant is what you buy and not store
        m.addConstr(gp.quicksum(planted[i][j] for j in crops) <= gp.quicksum(soilPrepped[i][x][y]
                                for x in range(copy.shape[0]) 
                                for y in range(copy.shape[1])))
        
        m.addConstr(gp.quicksum(sprinklerPutHere[i][x][y] for x in range(copy.shape[0]) for y in range(copy.shape[1])) 
                    <= sprinklersPurchased[i])
        m.addConstr(gp.quicksum(sprinklerCovered[i][x][y] for x in range(copy.shape[0]) for y in range(copy.shape[1]) )
                    <= 4 * sprinklersOwned[i])
        
        m.addConstr(sprinklersOwned[i] == sprinklersPurchased[i])
        
        m.addConstr(gp.quicksum(occupied[i][x][y] for x in range(copy.shape[0]) for y in range(copy.shape[1]))
                    == 0)
        
        for x in range(copy.shape[0]):
            for y in range(copy.shape[1]):
                if(x==0 and y ==0):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x+1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y+1] == 1)
                elif(x==0 and y == copy.shape[1]-1):
                    m.addConstr(sprinklerPutHere[i][x][y] == 0)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x+1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y-1] == 1)
                elif(x == 0 and y > 0 and y != copy.shape[1]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x+1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y-1] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y+1] == 1)
                    
                
                elif(y == 0 and x > 0 and x != copy.shape[0]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x+1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x-1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y+1] == 1)
                
                elif(y == 0 and x == copy.shape[0]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x-1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y+1] == 1)
                
                elif(x>0 and y > 0 and x != copy.shape[0]-1 and y != copy.shape[1]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x-1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x+1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y-1] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y+1] == 1)
                
                elif(x == copy.shape[0]-1 and y==copy.shape[1]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x-1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y-1] == 1)
                    
                elif(x == copy.shape[0]-1 and y > 0 and y != copy.shape[1]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x-1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y-1] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y+1] == 1)
                
                elif(y == copy.shape[1]-1 and x > 0 and x != copy.shape[0]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x+1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x-1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y-1] == 1)
                
                
                m.addGenConstrAnd(no_water_needed[i][x][y], [sprinklerCovered[i][x][y], occupied[i+1][x][y]])
                
                
                if(i + 1 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['blue_jaz'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['kale'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['parsnip'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['potato'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['tulip'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['unmilled_rice'], 1, occupied[i+1][x][y] == 1)

                if(i + 2 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['blue_jaz'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['kale'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['parsnip'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['potato'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['tulip'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['unmilled_rice'], 1, occupied[i+2][x][y] == 1)

                if(i + 3 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['blue_jaz'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['kale'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['parsnip'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['potato'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['tulip'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['unmilled_rice'], 1, occupied[i+3][x][y] == 1)

                if(i + 4 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['blue_jaz'], 1, occupied[i+4][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+4][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['kale'], 1, occupied[i+4][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['potato'], 1, occupied[i+4][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+4][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['tulip'], 1, occupied[i+4][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['unmilled_rice'], 1, occupied[i+4][x][y] == 1)

                if(i + 5 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['blue_jaz'], 1, occupied[i+5][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+5][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['kale'], 1, occupied[i+5][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['potato'], 1, occupied[i+5][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+5][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['tulip'], 1, occupied[i+5][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['unmilled_rice'], 1, occupied[i+5][x][y] == 1)

                if(i + 6 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['blue_jaz'], 1, occupied[i+6][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+6][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+6][x][y] == 1)

                if(i+7 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+7][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+7][x][y] == 1)
                if(i + 8 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+8][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+8][x][y] == 1)
                if(i + 9 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+9][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+9][x][y] == 1)
                if(i + 10 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+10][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+10][x][y] == 1)
                if(i+11 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+11][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+11][x][y] == 1)
                if(i + 12 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+12][x][y] == 1)
        
        
        
        for j in crops:
            m.addConstr(planted[i][j] == gp.quicksum(didPlant[i][x][y][j] 
                                                     for x in range(copy.shape[0])
                                                    for y in range(copy.shape[1])))
            for x in range(copy.shape[0]):
                for y in range(copy.shape[1]):
                    m.addGenConstrIndicator(soilPrepped[i][x][y], 0, didPlant[i][x][y][j] == 0)
                    
            m.addConstr(stored[i][j] == purchase[i][j] - planted[i][j] + starting_parsnips[i][j])
            m.addConstr(planted[i][j] == purchase[i][j] - stored[i][j] + starting_parsnips[i][j])
            m.addConstr(harvested[i][j] == 0)
            
    else:
        m.addConstr(sprinklersOwned[i] == sprinklersOwned[i-1] + sprinklersPurchased[i])
        m.addConstr(gp.quicksum(starting_parsnips[i][j] for j in crops) == 0)
        m.addConstr(money_in_days[i] == money_in_days[i-1] - gp.quicksum(purchase[i][j]*cost[j] for j in crops)
                   + gp.quicksum(harvested[i][j] * sell_price[j] for j in crops)
                    - 1125 * sprinklersPurchased[i])
        
        m.addGenConstrIndicator(bought_plants_today[i], 0, gp.quicksum(purchase[i][j] for j in crops) == 0)
        m.addGenConstrIndicator(bought_plants_today[i], 1, gp.quicksum(purchase[i][j] for j in crops) >= 1)
        m.addGenConstrIndicator(bought_sprinkler_today[i], 0, sprinklersPurchased[i] == 0)
        m.addGenConstrIndicator(bought_sprinkler_today[i], 1, sprinklersPurchased[i] >= 1)
        
        ## added in the regrow crops of greenBean and strawberry
        m.addConstr(gp.quicksum(purchase[i][j] for j in crops) <= money_in_days[i] + 
                    gp.quicksum(harvested[i][j] for j in crops))
        
        m.addConstr(amount_growing[i] == gp.quicksum(gp.quicksum(planted[k][j] - harvested[k][j] for j in crops) 
                                                     for k in range(i))
                    + gp.quicksum(greenBeanPlantedHere[i][x][y]
                                  for x in range(copy.shape[0])
                                  for y in range(copy.shape[1])) 
                    + gp.quicksum(strawberryPlantedHere[i][x][y] 
                                 for x in range(copy.shape[0])
                                 for y in range(copy.shape[1])))
        
        m.addConstr(gp.quicksum(occupied[i][x][y] for x in range(copy.shape[0]) for y in range(copy.shape[1]))
                   == gp.quicksum(occupied[i-1][x][y] for x in range(copy.shape[0]) for y in range(copy.shape[1]))
                    + gp.quicksum(didPlant[i-1][x][y][j] for x in range(copy.shape[0]) for y in range(copy.shape[1]) for j in crops)
                    - gp.quicksum(harvested[i][j] for j in crops)
                   )
        
        
        
        # for some reason it wasn't giving water to the ones we planted so i forced it to here
        m.addConstr(gp.quicksum(soilPrepped[i][x][y] * 2
                                for x in range(copy.shape[0])
                                for y in range(copy.shape[1])) -
                    gp.quicksum(soilPrepped[i - 1][x][y] * 2
                                for x in range(copy.shape[0])
                                for y in range(copy.shape[1]))
                    + 2 * amount_growing[i] 
                    + 2 * gp.quicksum(planted[i][j] for j in crops)
                    - 2 * gp.quicksum(no_water_needed[i][x][y] 
                                  for x in range(copy.shape[0])
                                  for y in range(copy.shape[1]))
                    == energySpent[i])
        
        m.addConstr(time_spent[i] == gp.quicksum(soilPrepped[i][x][y] * 2
                                for x in range(copy.shape[0])
                                for y in range(copy.shape[1]))
                    - 2 * gp.quicksum(soilPrepped[i - 1][x][y]
                                for x in range(copy.shape[0])
                                for y in range(copy.shape[1]))
                    + 3 * amount_growing[i]
                    + 4 * gp.quicksum(planted[i][j] for j in crops)
                    + 4 * gp.quicksum(harvested[i][j] for j in crops)
                    - 4 * gp.quicksum(no_water_needed[i][x][y]
                                      for x in range(copy.shape[0])
                                      for y in range(copy.shape[1]))
                    + 24 * bought_plants_today[i]
                   + 40 * bought_sprinkler_today[i])
                   
        
        
        m.addConstr(gp.quicksum(sprinklerPutHere[i][x][y] for x in range(copy.shape[0]) for y in range(copy.shape[1])) 
                    <= sprinklersOwned[i])
        
        m.addConstr(gp.quicksum(sprinklerCovered[i][x][y] for x in range(copy.shape[0]) for y in range(copy.shape[1]) )
                    <= 4 * sprinklersOwned[i])
        
        m.addConstr(time_spent[i] <= 810 + energy_penalty[i])
        
        
        m.addConstr(energySpent[i] <= 270 - 0.1125 * energy_penalty[i-1])
        for x in range(copy.shape[0]):
            for y in range(copy.shape[1]):
                m.addGenConstrIndicator(soilPrepped[i-1][x][y], 1, soilPrepped[i][x][y]==1)
                m.addGenConstrIndicator(sprinklerPutHere[i-1][x][y], 1, sprinklerPutHere[i][x][y] == 1)
                if(x==0 and y ==0):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x+1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y+1] == 1)
                elif(x==0 and y == copy.shape[1]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x+1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y-1] == 1)
                elif(x == 0 and y > 0 and y != copy.shape[1]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x+1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y-1] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y+1] == 1)
                    
                elif(y == 0 and x > 0 and x != copy.shape[0]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x+1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x-1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y+1] == 1)
                
                elif(y == 0 and x == copy.shape[0]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x-1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y+1] == 1)
                
                elif(x>0 and y > 0 and x != copy.shape[0]-1 and y != copy.shape[1]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x-1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x+1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y-1] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y+1] == 1)
                
                elif(x == copy.shape[0]-1 and y==copy.shape[1]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x-1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y-1] == 1)
                    
                elif(x == copy.shape[0]-1 and y > 0 and y != copy.shape[1]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x-1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y-1] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y+1] == 1)
                
                elif(y == copy.shape[1]-1 and x > 0 and x != copy.shape[0]-1):
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x+1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x-1][y] == 1)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y], 1, sprinklerCovered[i][x][y-1] == 1)
                
                m.addGenConstrAnd(no_water_needed[i][x][y], [sprinklerCovered[i][x][y], occupied[i][x][y]])
                m.addGenConstrIndicator(occupied[i][x][y], 1, sprinklerPutHere[i][x][y] == 0)
                
                
                
                ## if a regrow crop was ever planted then it's always occupied
                m.addGenConstrIndicator(greenBeanPlantedHere[i][x][y], 0, gp.quicksum(didPlant[I][x][y]['green_bean']
                                                                                     for I in range(i)) == 0)
                m.addGenConstrIndicator(strawberryPlantedHere[i][x][y], 0, gp.quicksum(didPlant[I][x][y]['strawberry']
                                                                                     for I in range(i)) == 0)
                m.addGenConstrIndicator(greenBeanPlantedHere[i][x][y], 1, gp.quicksum(didPlant[I][x][y]['green_bean']
                                                                                     for I in range(i)) == 1)
                m.addGenConstrIndicator(strawberryPlantedHere[i][x][y], 1, gp.quicksum(didPlant[I][x][y]['strawberry']
                                                                                     for I in range(i)) == 1)
                
                if(i + 1 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['blue_jaz'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['kale'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['parsnip'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['potato'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['tulip'], 1, occupied[i+1][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['unmilled_rice'], 1, occupied[i+1][x][y] == 1)

                if(i + 2 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['blue_jaz'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['kale'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['parsnip'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['potato'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['tulip'], 1, occupied[i+2][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['unmilled_rice'], 1, occupied[i+2][x][y] == 1)

                if(i + 3 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['blue_jaz'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['kale'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['parsnip'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['potato'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['tulip'], 1, occupied[i+3][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['unmilled_rice'], 1, occupied[i+3][x][y] == 1)

                if(i + 4 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['blue_jaz'], 1, occupied[i+4][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+4][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['kale'], 1, occupied[i+4][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['potato'], 1, occupied[i+4][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+4][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['tulip'], 1, occupied[i+4][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['unmilled_rice'], 1, occupied[i+4][x][y] == 1)

                if(i + 5 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['blue_jaz'], 1, occupied[i+5][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+5][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['kale'], 1, occupied[i+5][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['potato'], 1, occupied[i+5][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+5][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['tulip'], 1, occupied[i+5][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['unmilled_rice'], 1, occupied[i+5][x][y] == 1)

                if(i + 6 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['blue_jaz'], 1, occupied[i+6][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+6][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+6][x][y] == 1)

                if(i+7 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+7][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+7][x][y] == 1)
                if(i + 8 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+8][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+8][x][y] == 1)
                if(i + 9 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+9][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+9][x][y] == 1)
                if(i + 10 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+10][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+10][x][y] == 1)
                if(i+11 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['cauliflower'], 1, occupied[i+11][x][y] == 1)
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+11][x][y] == 1)
                if(i + 12 < days):
                    m.addGenConstrIndicator(didPlant[i][x][y]['rhubarb'], 1, occupied[i+12][x][y] == 1)

        for j in crops:
            m.addConstr(stored[i][j] == stored[i-1][j] + purchase[i][j] - planted[i][j])
            m.addConstr(planted[i][j] <= stored[i-1][j] + purchase[i][j])
            m.addConstr(planted[i][j] == gp.quicksum(didPlant[i][x][y][j] 
                                                     for x in range(copy.shape[0]) for y in range(copy.shape[1])))
            for x in range(copy.shape[0]):
                for y in range(copy.shape[1]):
                    ## can't plant if it's occupied or the soil is not prepped or sprinkler is there
                    m.addGenConstrIndicator(occupied[i][x][y], 1, didPlant[i][x][y][j] == 0)
                    m.addGenConstrIndicator(soilPrepped[i][x][y], 0, didPlant[i][x][y][j] == 0)
                    m.addGenConstrIndicator(sprinklerPutHere[i][x][y],1, didPlant[i][x][y][j] == 0)
                    
                    
            if(yeild_time[j] > i):
                m.addConstr(harvested[i][j] == 0)
            else:
                if(regrow[j] is not None):
                    m.addConstr(harvested[i][j] == gp.quicksum(planted[i-yeild_time[j]-k*regrow[j]][j] 
                                                               for k in range(max_range_regrow(i,j))))
                else:
                    m.addConstr(harvested[i][j] == (planted[i-yeild_time[j]][j]))

m.optimize()
write_to_gruobi_txt(m)
for v in m.getVars():
    
    if('money' in v.varName or 'time' in v.varName):
        print('%s %g' % (v.varName, v.x))
    
    
    

Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter MIPGap to 0.011
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 1457 rows, 401968 columns and 754333 nonzeros
Model fingerprint: 0x57630049
Model has 1993398 general constraints
Variable types: 23940 continuous, 378028 integer (376768 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 8e+02]
Presolve removed 315 rows and 9803 columns (presolve time = 6s) ...
Presolve removed 45574 rows and 55765 columns (presolve time = 10s) ...
Presolve removed 119607 rows and 129798 columns (presolve time = 15s) ...
Presolve removed 119612 rows and 132338 columns (presolve time = 20s) ...
Presolve removed 119612 rows and 2076749 columns (presolve time = 3000s) ...
Presolve removed 119612 rows and 2217109 co

  66   6.50687318e+03  1.95606766e+05  2.06e+01 1.81e-12  1.40e-01  3996s
  67   7.20733864e+03  1.87895208e+05  1.79e+01 3.78e-12  1.28e-01  4007s
  68   7.19032950e+03  1.83222489e+05  1.77e+01 3.15e-12  1.25e-01  4018s
  69   7.40865978e+03  1.73496911e+05  1.68e+01 3.57e-12  1.17e-01  4029s
  70   7.69131328e+03  1.64430783e+05  1.54e+01 2.82e-12  1.09e-01  4040s
  71   7.73257348e+03  1.51951543e+05  1.35e+01 2.37e-12  9.77e-02  4051s
  72   8.50043599e+03  1.41021920e+05  9.38e+00 3.08e-12  8.17e-02  4063s
  73   8.79135600e+03  1.18721703e+05  8.13e+00 1.78e-12  6.75e-02  4074s
  74   9.54322353e+03  9.45557370e+04  5.50e+00 1.64e-12  4.95e-02  4085s
  75   9.90039609e+03  8.39518809e+04  4.19e+00 1.71e-12  4.16e-02  4095s
  76   1.03358969e+04  7.20474110e+04  3.39e+00 1.59e-12  3.42e-02  4107s
  77   1.09281582e+04  5.95304292e+04  2.34e+00 9.13e-13  2.60e-02  4118s
  78   1.13324260e+04  5.06691394e+04  1.74e+00 1.25e-12  2.07e-02  4129s
  79   1.16372376e+04  4.55136273e+04 

In [ ]:
def write_to_gruobi_txt(m):
    file = open('gurobi_out.txt', 'w')
    varInfo = [(v.varName, v.X) for v in m.getVars()]
    varInfo = [f'{v[0]} {v[1]} \n' for v in varInfo]
    file.writelines(varInfo)
    file.close()

In [424]:
print(copy.shape)

(30, 40)
